In [109]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

In [110]:
model = read_sbml_model('../data/models/iMM904.xml')
model_new = model

In [111]:
# Reaction 1 (HA-TAL):L-tyrosine <=> (E).4.coumarate +   NH4+
new_reaction1 = Reaction('HA-TAL') # 3-(o)xopropanoate:(N)ADP+ (o)xidoreductase
coumarate4 = Metabolite(id='coumarate4', compartment='c')
new_reaction1.add_metabolites({model_new.metabolites.tyr__L_c: -1, # Search L-tyrisine ID
                               coumarate4: 1, # 3-Oxopropanoate
                               model_new.metabolites.nh4_c: 1, # Search fpr NH4 ID
                              })
model_new.add_reactions([new_reaction1])

In [116]:
model_new.metabolites.coumarate4

Metabolite identifier,coumarate4
Name,
Memory address,0x1e0c0c54400
Formula,None
Compartment,c
In 1 reaction(s),HA-TAL


In [117]:
new_reaction2 = Reaction('AT-4CL1')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction2.add_metabolites({model_new.metabolites.coumarate4: -1,
                               model_new.metabolites.atp_c: -1,
                               model_new.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model_new.metabolites.amp_c: 1,
                               model_new.metabolites.ppi_c: 1,
                              })
model_new.add_reactions([new_reaction2])

In [118]:
new_reaction3 = Reaction('VVVST1')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction3.add_metabolites({model_new.metabolites.e4_coumaroyl_coa: -1,
                               model_new.metabolites.h_c: -3,
                               model_new.metabolites.malcoa_c: -3,
                               model_new.metabolites.co2_c: 4,
                               model_new.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model_new.add_reactions([new_reaction3])

In [119]:
model_new.reactions.get_by_id('VVVST1')

Reaction identifier,VVVST1
Name,
Memory address,0x1e0c434c6d0
Stoichiometry,e4_coumaroyl_coa + 3 h_c + 3 malcoa_c --> 4 co2_c + 4 coa_c + trans_resv + 3 H+ + 3 Malonyl CoA C24H33N7O19P3S --> 4 CO2 CO2 + 4 Coenzyme A +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [120]:
from cobra.io import write_sbml_model
write_sbml_model(model_new,'data/models/yeast_resv.xml')

#### how much resveratrol is produced?

In [121]:
#model_new.add_boundary(model_new.metabolites.trans_resv, type='demand')

In [122]:
with model_new:
    model_new.objective=model_new.reactions.VVVST1
    lets_produce_resv= model_new.optimize().objective_value
lets_produce_resv

9.281337700198714e-32

## Model 2: Resveratrol from coumaric acid from https://doi.org/10.1128/AEM.00609-06

In [182]:
model = read_sbml_model('../data/models/iMM904.xml')
model_new2=model

In [183]:
new_reaction2_1 = Reaction('AT-4CL3')
coumarate4 = Metabolite(id='coumarate4', compartment='c')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction2_1.add_metabolites({coumarate4: -1,
                               model_new2.metabolites.atp_c: -1,
                               model_new2.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model_new2.metabolites.amp_c: 1,
                               model_new2.metabolites.ppi_c: 1,
                              })

model_new2.add_reactions([new_reaction2_1])

In [184]:
model_new2.metabolites.coumarate4

Metabolite identifier,coumarate4
Name,
Memory address,0x1e0c8e28c40
Formula,None
Compartment,c
In 1 reaction(s),AT-4CL3


In [185]:
new_reaction2_2 = Reaction('STS')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction2_2.add_metabolites({model_new2.metabolites.e4_coumaroyl_coa: -1,
                               model_new2.metabolites.h_c: -3,
                               model_new2.metabolites.malcoa_c: -3,
                               model_new2.metabolites.co2_c: 4,
                               model_new2.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model_new2.add_reactions([new_reaction2_2])



In [186]:
from cobra.io import write_sbml_model
write_sbml_model(model_new,'data/models/yeast_resv_coumaricacid.xml')

In [187]:
#how much resveratrol is produced?
model_new2.add_boundary(model_new.metabolites.trans_resv, type='demand')
with model_new2:
    model_new2.objective=model_new2.reactions.STS
    lets_produce_resv= model_new2.optimize().objective_value
lets_produce_resv

-3.389454704016765e-33

## Model 3: Resveratrol from Phenylalanine (https://doi.org/10.1016/j.ymben.2009.07.004)


In [175]:
model = read_sbml_model('../data/models/iMM904.xml')
model_new3 = model

In [176]:
new_reaction3_1 = Reaction('PAL3')
e_cinnamate_c = Metabolite(id='e_cinnamate_c', compartment='c')
new_reaction3_1.add_metabolites({model_new3.metabolites.phe__L_c: -1,
                               e_cinnamate_c: 1,
                               model_new3.metabolites.nh4_c: 1,
                              })
model_new3.add_reactions([new_reaction3_1])

In [177]:
new_reaction3_2 = Reaction('C4H') 
coumarate4 = Metabolite(id='coumarate4', compartment='c')
new_reaction3_2.add_metabolites({model_new3.metabolites.e_cinnamate_c: -1,
                               model_new3.metabolites.fmnh2_c: -1,
                               model_new3.metabolites.o2_c: -1,
                               coumarate4: 1, 
                               model_new3.metabolites.h2o_c: 1,
                               model_new3.metabolites.h_c: 1,
                               model_new3.metabolites.fmn_c: 1,
                              })
model_new3.add_reactions([new_reaction3_2])

In [178]:
new_reaction3_3 = Reaction('AT-4CL2')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction3_3.add_metabolites({model_new.metabolites.coumarate4: -1,
                               model_new.metabolites.atp_c: -1,
                               model_new.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model_new.metabolites.amp_c: 1,
                               model_new.metabolites.ppi_c: 1,
                              })
model_new3.add_reactions([new_reaction3_3])

In [179]:
new_reaction3_4 = Reaction('STS')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction3_4.add_metabolites({model_new.metabolites.e4_coumaroyl_coa: -1,
                               model_new.metabolites.h_c: -3,
                               model_new.metabolites.malcoa_c: -3,
                               model_new.metabolites.co2_c: 4,
                               model_new.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model_new3.add_reactions([new_reaction3_4])

In [180]:
from cobra.io import write_sbml_model
write_sbml_model(model_new,'data/models/yeast_resv_phenylalanine.xml')

In [181]:
#how much resveratrol is produced?
model_new3.add_boundary(model_new.metabolites.trans_resv, type='demand')
with model_new3:
    model_new3.objective=model_new3.reactions.STS
    lets_produce_resv= model_new3.optimize().objective_value
lets_produce_resv

1.0749185667752432